In [1]:
import pandas as pd
import numpy as np
import scipy.constants as sc
from scipy.integrate import quad

In [9]:
df = pd.read_excel(r"coefficient2.xlsx")
new_df = pd.DataFrame({"Frequency": df.loc[1:, 'Unnamed: 0'].apply(lambda x:float(x))}, index=range(1,11))

In [4]:
k = sc.value(u"Boltzmann constant")  # Boltzman constant in J/K
h = sc.value(u"Planck constant")           # Planck's Constant in J/Hz
f = df.loc[1:,"Unnamed: 0"].apply(lambda x:float(x))*10**12     # Frequency in Hz
n_skin = df.loc[1:, "skin"].apply(lambda x:float(x))    # refractive index for skin
a_skin = df.loc[1:, "skin.1"].apply(lambda x:float(x))  # absorption coefficient for skin
d = 1 * 10**(-3)             # Distance in meter

# Background Noise by Planck's Function
planck_func = [((2*sc.pi*h*(f[i])**3)/((sc.c)**2))*1/(np.exp((h*f[i])/(k*310))) for i in range(1,len(f)+1)]

# Background Noise power spectral density
bg_noise_skin = ((planck_func)*(sc.c/4*sc.pi*f*n_skin)**2).apply(lambda x:float(x))

# Noise on skintissue medium
noise_skin = (bg_noise_skin*(1-np.exp(-(a_skin*d*10**3))) * (sc.c/4*sc.pi*n_skin*f*d)**2).apply(lambda x:float(x))   

# molecular absorption noise for skin
abs_noise_skin = bg_noise_skin + noise_skin

In [5]:
new_df["Skin (Refractive Index, n)"] = df.loc[1:,"skin"].apply(lambda x:float(x))
new_df["Skin (Absorption Coefficient, \u03B1)"] = df.loc[1:,"skin.1"].apply(lambda x:float(x))
new_df["Fat (Refractive Index, n)"] = df.loc[1:,"Fat"].apply(lambda x:float(x))
new_df["Fat (Absorption Coefficient, \u03B1)"] = df.loc[1:,"Fat.1"].apply(lambda x:float(x))
new_df["Plank's Function B(T,f)"] = planck_func
new_df["Background Noise power spectral density S(f) for Skin"] = bg_noise_skin
new_df["Noise S_s(d,f) for Skin"] = noise_skin
new_df["Molecular absorption noise S_N(d,f) for Skin"] = abs_noise_skin

In [6]:
noise = []
for i in range(1, len(a_skin)+1):
    func = lambda  freq:k*310* (1-np.exp(-a_skin[i]*d*10**3))* (sc.c/(freq*np.sqrt(4*sc.pi)))**2
    val, err = quad(func, 0, planck_func[i-1])
    noise.append(val)

<ipython-input-6-ee06341cb88a>:4: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  val, err = quad(func, 0, planck_func[i-1])


In [7]:
noise_dB = [10*np.log10(-noise[i]) for i in range(len(noise))]     # Noise in dB
new_df["Noise Power P_N in dB"] = noise_dB
new_df.to_excel("noise_data.xlsx")
new_df

,Frequency,"Skin (Refractive Index, n)","Skin (Absorption Coefficient, α)","Fat (Refractive Index, n)","Fat (Absorption Coefficient, α)","Plank's Function B(T,f)",Background Noise power spectral density S(f) for Skin,"Noise S_s(d,f) for Skin","Molecular absorption noise S_N(d,f) for Skin",Noise Power P_N in dB
1,0.2,2.487803,7.240005,2.0305,4.0489,3.592837e-16,4.931177e+24,6.763198e+58,6.763198e+58,109.301274
2,0.4,2.246139,9.490481,1.9319,5.8964,2.786637e-15,1.247082e+26,5.580543e+60,5.580543e+60,100.407632
3,0.6,2.165657,11.736955,1.8659,8.0317,9.118161e-15,8.535132e+26,7.989319e+61,7.989319e+61,95.259658
4,0.8,2.111387,14.240531,1.8101,10.0315,2.095446e-14,3.314463e+27,5.242634e+62,5.242634e+62,91.645998
5,1.0,2.065012,16.846053,1.7649,11.7077,3.967889e-14,9.380497e+27,2.217646e+63,2.217646e+63,88.873170
6,1.2,2.023375,19.394905,1.7295,13.0432,6.647468e-14,2.172664e+28,7.101151e+63,7.101151e+63,86.632203
7,1.4,1.986050,21.781510,1.7021,14.0876,1.023410e-13,4.386392e+28,1.880032e+64,1.880032e+64,84.758269
8,1.6,1.952969,23.951465,1.6809,14.8994,1.481080e-13,8.017339e+28,4.339923e+64,4.339923e+64,83.152981
9,1.8,1.923947,25.887087,1.6644,15.5355,2.044509e-13,1.359382e+29,9.038453e+64,9.038453e+64,81.752875
10,2.0,1.898668,27.593195,1.6514,16.0376,2.719033e-13,2.173669e+29,1.737691e+65,1.737691e+65,80.514620
